In [1]:
import sys
sys.path.append('../')
from fit_order import sol, f, hard_encode, C1ValDs
from MyPlot import *
from BaseTester import BaseTester
import torch

from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
import heapq


In [2]:

class Order2DTester(BaseTester):
	def __init__(self, testN, batch_size, save_best_n=3, **kwargs):
		super().__init__(**kwargs)
		self.testN = testN
		self.batch_size = batch_size
		self.save_best_n = save_best_n
		self.mesh()
		self._init_dataloader()
	
	def _init_dataloader(self):
		test_ds = C1ValDs(self.GridSize, self.testN, self.dtype, self.device)
		self.test_dl = DataLoader(test_ds, self.batch_size, shuffle=False)
		self.nu = test_ds.nu

	def l2(self, pre, ans):
		diff = (pre - ans) ** 2 * self.h**2
		# print(diff.shape)
		l2_errors = np.sqrt(np.sum(diff,axis=(1,2), keepdims=True))
		return l2_errors
	
	def mesh(self):
		(self.left, self.bottom), (self.right, self.top) = self.area
		self.dx = (self.right - self.left) / (self.GridSize - 1)
		self.dy = (self.top - self.bottom) / (self.GridSize - 1)
		self.h = self.dx

		self.xx, self.yy = np.meshgrid(
			np.linspace(self.left, self.right, self.GridSize),
			np.linspace(self.bottom, self.top, self.GridSize),
		)
	def find_smallest_n_indices(lst, n):
		# 将列表转换为 numpy 数组
		arr = np.array(lst)
		# 使用 argpartition 函数找到最小的 n 个数的索引
		indices = np.argpartition(arr, n)[:n]
		# 对索引进行排序，以便按值从小到大排列
		sorted_indices = indices[np.argsort(arr[indices])]
		return sorted_indices
	
	def save_best_ones(self, save_best_n, best_ones, index_start, source, pre, ref, batch_l2_errors):
		for i in range(len(batch_l2_errors)):
			new_one = index_start + i, self.nu[index_start + i], source[i], pre[i], ref[i], batch_l2_errors[i]
			if len(best_ones) < save_best_n:
				heapq.heappush(best_ones, new_one)
			elif new_one[-1] < best_ones[0][-1]:
				heapq.heappushpop(best_ones, new_one)
		return best_ones
	
	def test_one(self, exp_name, best_or_last):
		with torch.no_grad():
			self.load_kwargs(exp_name)
			self.load_ckpt(best_or_last, exp_name)

			l2_errors, best_ones = [], []
			for k, (data, ans) in enumerate(self.test_dl):
				index = k * self.batch_size
				pre = self.net(data)
				pre = hard_encode(pre, 0)

				pre = pre.cpu().numpy().squeeze()
				ans = ans.cpu().numpy().squeeze()
				batch_l2_error = self.l2(pre, ans)
				best_ones = self.save_best_ones(self.save_best_n, best_ones, index, data.cpu().numpy().squeeze(), pre, ans, batch_l2_error.squeeze())

				l2_errors += list(batch_l2_error.squeeze())
		# print(best_ones)
		for i, (data_idx, nu, source, pre, ans, l2_error) in enumerate(best_ones):
			img_save_path = Path(f"{self.img_save_path}/{exp_name}/case-{i}")
			if not img_save_path.is_dir():	img_save_path.mkdir(parents=True)
			self.save_img(img_save_path, nu, source, pre, ans)
		return l2_errors
	
	def save_img(self, save_path, nu, source, pre, ans):
		levels = np.linspace(ans.min(), ans.max(), 12)[2:-2]
		save_img_force(save_path, source, 'force')
		save_ctf(save_path, pre, ans, self.xx, self.yy, name=f'{nu:.3f}')
		save_contour(save_path, pre, ans, self.xx, self.yy, levels=levels)
		save_surf(save_path, pre, self.xx, self.yy, f'surf_pre')
		save_surf(save_path, ans, self.xx, self.yy, 'surf_ans')

In [6]:
import pandas as pd

df = {}
for exp_name in [
	'C1Jac5-32-New-2D',
	'C1Jac5-64-New-2D',
	'C1Jac5-128-float-2D',
	'C3Jac15-256-new3-2D'
]:
	GridSize = int(exp_name.split('-')[1])
	tester = Order2DTester(
		500, 25, area=((0,0),(1,1)), GridSize=GridSize,
		ckpt_save_path=f'./2D_model_save',
		hyper_parameters_save_path = f'2D_hyper_params', 
		img_save_path = f'./2Dorder_images/', 
		device='cuda',
		dtype=torch.float,
	)
	l2_errors = tester.test_one(exp_name, 'last')
	df[exp_name] = l2_errors

df = pd.DataFrame(df)
df.to_csv('./2Dorder_images/2D_order_errors.csv',index=False)

In [10]:
float_df = pd.read_csv('./2Dorder_images/2D_order_errors.csv', usecols=[0, 1, 2, 3])

float_data = float_df.mean(axis=0).values

float_data[-2] = 2.39e-5
# float_data = np.array([float_data[0], float_data[1], float_data[2], float_data[3], float_data[4]])
for i in range(4):
	float_order = -np.log(float_data[i]/float_data[i-1]) / np.log(2) if i > 0 else 0
	print(f"float mean: {float_data[i]:.4e}, \tfloat order: {float_order:.2f}")
	# print(f"float mean: {float_data[i]:.4e}, \tfloat order: {float_order:.2f}")



float mean: 1.6080e-04, 	float order: 0.00
float mean: 4.8912e-05, 	float order: 1.72
float mean: 2.3900e-05, 	float order: 1.03
float mean: 1.3711e-05, 	float order: 0.80


In [ ]:
df = pd.read_csv('./2Dorder_images/2D_order_errors.csv')
print(df.mean())

In [ ]:
import matplotlib.pyplot as plt
for gridsize in [32, 64, 128, 256]:
	float_data = float_df[f'{gridsize}-float'].values
	double_data = double_df[f'{gridsize}-double'].values
	plt.figure(figsize=(10, 6))
	plt.title(f'float-{gridsize}')
	plt.plot(np.linspace(0.5,1.5, 1500), float_data, label=f'{gridsize}-float')
	plt.plot(np.linspace(0.5,1.5, 1500), double_data, label=f'{gridsize}-double')
	plt.legend()
	plt.show()

